# Hate Speech Detector 2.0
---
**Vulgar Phrases Dictionary**

A dictionary of polish vulgar words (i.e. swear or abusive) and phrases. Extending the dictionary using polish WordNet and researching within which wider phrases do the vulgar phrases appear.

In [1]:
import re
import numpy as np
import os
import csv
import pickle

from pyplwnxml import PlwnxmlParser
import morfeusz2

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

## Loading the data
The three data sources have been found:
1. Polish vulgar words [Marcinmazurek.com.pl](http://marcinmazurek.com.pl/polskie-wulgaryzmy)
2. Polish vulgar words [Github.com](https://github.com/coldner/wulgaryzmy)
3. Polish vulgar words & phrases [Pl.wiktionary.org](https://pl.wiktionary.org/wiki/Indeks:Polski_-_Wulgaryzmy)

In [2]:
with open('data/vulgars/polish_vulgar_words.txt', 'r') as f:
    text = f.read()

vulgar1 = np.array(re.findall(r'\'([A-Za-z0-9_\./\\-]*)\'', text))
f'Set 1: {len(vulgar1)} examples.'

'Set 1: 413 examples.'

In [3]:
with open('data/vulgars/polish_vulgar_words_github.txt', 'r') as f:
    text = f.read()

vulgar2 = np.array(re.findall(r'\"([A-Za-z0-9_\./\\-]*)\"', text))
f'Set 2: {len(vulgar2)} examples.'

'Set 2: 464 examples.'

In [4]:
with open('data/vulgars/polish_vulgar_phrases.txt', 'r') as f:
    text = f.read()
text = text.replace('\n', ' • ')

vulgar3 = np.array(text.split(' • '))
f'Set 3: {len(vulgar3)} examples.'

'Set 3: 423 examples.'

In [5]:
vulgar_words = np.union1d(vulgar1, vulgar2)
vulgar_words = np.union1d(vulgar_words, vulgar3)
f'Total: {len(vulgar_words)} distinct examples.'

'Total: 846 distinct examples.'

In [6]:
with open('data/vulgars/polish_vulgars.txt', 'w') as f:
    f.write(';'.join(vulgar_words))

## Polish WordNet 3.1
---
Looking for synonyms of the hateful words using the polish WordNet.

In [7]:
pl_wordnet = PlwnxmlParser('models/plwordnet_3_0/plwordnet-3.0.xml').read_wordnet()

In [8]:
for lu in pl_wordnet.lemma("zły")[0].synsets[0].lexical_units:
    print(lu.name)

zły
niedobry


In [9]:
for lu in pl_wordnet.lemma("piec")[0].synsets[0].lexical_units:
    print(lu.name)

piec
piec centralny


In [10]:
pl_wordnet.lemma("drobiazg")

[<LexicalUnit - id:1357,name:drobiazg,pos:PartOfSpeech.NOUN,domain:Domain.ATTRIBUTE,sentiment:None>,
 <LexicalUnit - id:109778,name:drobiazg,pos:PartOfSpeech.NOUN,domain:Domain.HUMAN_PRODUCTS_NAMES,sentiment:None>,
 <LexicalUnit - id:109779,name:drobiazg,pos:PartOfSpeech.NOUN,domain:Domain.COMMUNICATION,sentiment:None>,
 <LexicalUnit - id:109781,name:drobiazg,pos:PartOfSpeech.NOUN,domain:Domain.GROUP,sentiment:None>]

In [11]:
vulgar_words_ext_neg, vulgar_words_ext_neu, vulgar_words_ext_pos = list([]), list([]), list([])
for vulgar_word in vulgar_words:
    for lemm in pl_wordnet.lemma(vulgar_word):
        for synset in lemm.synsets:
            for lu in synset.lexical_units:
                if lu.sentiment in ['- m', '- s'] and\
                    lu.name not in vulgar_words and lu.name not in vulgar_words_ext_neg:
                        vulgar_words_ext_neg.append(lu.name)
                if lu.sentiment in ['0', None] and\
                    lu.name not in vulgar_words and lu.name not in vulgar_words_ext_neu:
                        vulgar_words_ext_neu.append(lu.name)
                if lu.sentiment in ['+ m', '+ s'] and\
                    lu.name not in vulgar_words and lu.name not in vulgar_words_ext_pos:
                        vulgar_words_ext_pos.append(lu.name)

In [12]:
f'Additional negative sentiment words: {len(vulgar_words_ext_neg)}.'

'Additional negative sentiment words: 0.'

In [13]:
f'Additional neutral sentiment words: {len(vulgar_words_ext_neu)}.'

'Additional neutral sentiment words: 453.'

In [14]:
f'Additional positive sentiment words: {len(vulgar_words_ext_pos)}.'

'Additional positive sentiment words: 0.'

In [15]:
with open('data/vulgars/polish_additionalwn_vulgars.txt', 'w') as f:
    f.write(';'.join(vulgar_words_ext_neu))

## Vulgar phrases appearance analysis
---
Within which phrases do the vulgar words appear. Twitter scraped data. Morfeusz2 lemmatizer + Latent Dirichlet Allocation.

**First prepare Morfeusz2 lemmatizer**

In [16]:
morf = morfeusz2.Morfeusz()

In [17]:
text = ' Mam nadzieję że dzisiejsza akcja pań z opozycji na tyle wkr Kaczora, ' +\
       'ze dziś już policja nie będzie się certolić z tymi spędami'
print(text)
analysis = morf.analyse(text)
for i, j, interpretation in analysis:
    print(i, j, interpretation)

 Mam nadzieję że dzisiejsza akcja pań z opozycji na tyle wkr Kaczora, ze dziś już policja nie będzie się certolić z tymi spędami
0 1 ('Mam', 'mama', 'subst:pl:gen:f', ['nazwa_pospolita'], [])
0 1 ('Mam', 'mamić', 'impt:sg:sec:imperf', [], [])
0 1 ('Mam', 'mieć', 'fin:sg:pri:imperf', [], [])
1 2 ('nadzieję', 'nadzieja', 'subst:sg:acc:f', ['nazwa_pospolita'], [])
1 2 ('nadzieję', 'nadziać', 'fin:sg:pri:perf', [], [])
2 3 ('że', 'że:c', 'comp', [], [])
2 3 ('że', 'że:q', 'part', [], [])
3 4 ('dzisiejsza', 'dzisiejszy', 'adj:sg:nom.voc:f:pos', [], [])
4 5 ('akcja', 'akcja', 'subst:sg:nom:f', ['nazwa_pospolita'], [])
5 6 ('pań', 'pani', 'subst:pl:gen:f', ['nazwa_pospolita'], [])
6 7 ('z', 'z:p', 'prep:gen:nwok', [], [])
6 7 ('z', 'z:p', 'prep:inst:nwok', [], [])
6 7 ('z', 'z:q', 'part:nwok', [], [])
7 8 ('opozycji', 'opozycja', 'subst:sg:gen:f', ['nazwa_pospolita'], [])
7 8 ('opozycji', 'opozycja', 'subst:sg:dat.loc:f', ['nazwa_pospolita'], [])
7 8 ('opozycji', 'opozycja', 'subst:pl:gen:f',

In [18]:
def lemmatize_text(text):
    analysis = morf.analyse(text)
    lemmas = list([])
    
    i, j, interp = analysis[0]
    last_ij, last_lemma = (i, j), interp[1].split(':')[0]
    lemmas.append(last_lemma)
    
    for i, j, interp in analysis[1:]:
        lemma = interp[1].split(':')[0]
        if not (last_ij == (i, j) and last_lemma == lemma):
            lemmas.append(lemma)
        
        last_ij = (i, j)
        last_lemma = lemma
    
    return ' '.join(lemmas)

In [19]:
print(text)
lemmatize_text(text)

 Mam nadzieję że dzisiejsza akcja pań z opozycji na tyle wkr Kaczora, ze dziś już policja nie będzie się certolić z tymi spędami


'mama mamić mieć nadzieja nadziać że dzisiejszy akcja pani z opozycja na tył tyli tyle WKR kaczor Kaczor Kaczór , z dziś już policja nie on nie być się certolić z ten spęd'

**Second combine all vulgar pure texts and lemmatized texts into single columns.**

In [20]:
texts, texts_lemmatized = np.array([]), np.array([])

In [21]:
for file in tqdm(os.listdir('data/vulgars__texts')):
    with open(f'data/vulgars__texts/{file}', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        t = np.array([l for l in reader]).flatten()
        t_l = np.array([lemmatize_text(l) for l in t])
        
        texts = np.concatenate([texts, t])
        texts_lemmatized = np.concatenate([texts_lemmatized, t_l])

In [22]:
texts[:2]

array(['Nie rozumiem jednego . Na chuj Glodziowi potrzebna jest taka gabarytowo kobyła . Na bank papa Francisco wypieprzy go ze stanu duchownego za tuszowanie przestępstw seksualnych . Psy z ABW już węszą skąd miał kesz na te landarę.',
       'Nożeż qrwa nagła jej mać! Kiedy wreszcie wypieprzy z gwizdem tę imbecylkę Olejnik?!!!!! Zaprosiła (pozwoliła zaprosić) do swojej cipkokropki faszystę Krzysztofa JakaPięknaŁuna Bosaka i pozwala mu świrdolić, że wczoraj w Warszawie maszerowało "kilka tysięcy patriotów". '],
      dtype='<U292')

In [23]:
texts_lemmatized[:2]

array(['nie on nie rozumieć jedno jeden . na chuj Glodziowi potrzebny być taki taka gabarytowo kobyła . na bank papa papać Francisco wypieprzyć go on z stan duchowny za tuszować przestępstwo seksualny . pies z ABW już węszyć skąd miał mieć kesz na te ten ty landara .',
       'Nożeż qrwa nagły jej on mać ! Kieda kiedy wreszcie wypieprzyć z gwizd ten imbecylkę Olejnik ? ! ! ! ! ! zaprosić ( pozwolić zaprosić ) do swój cipkokropki faszysta Krzysztofa Krzysztof JakaPięknaŁuna bosak Bosak bosak i pozwalać mu on świrdolić , że wczoraj w warszawa Warszawa maszerować " kilka tysiąc patriota " .'],
      dtype='<U653')

**Third perform LDA algorithm and see results**

In [24]:
with open('data/other/polish_stopwords.txt', 'r') as f:\
    polish_stopwords = f.read().split('\n')[:-1]
polish_stopwords[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

In [25]:
cv = CountVectorizer(stop_words=polish_stopwords)
count_data = cv.fit_transform(texts)

In [26]:
def lda_topics(lda_model, lda_cv, n_words):
    words = lda_cv.get_feature_names()
    
    topics = list([' '.join([words[i] for i in topic.argsort()[:-n_words - 1:-1]])
                   for topic in lda_model.components_])
    
    return topics

In [27]:
N_TOPICS = 10
N_WORDS = 10

In [28]:
lda = LDA(n_components=N_TOPICS, n_jobs=-1)
lda.fit(count_data)
with open('models/lda/lda_vulg.pkl', 'wb') as f:
    pickle.dump([lda, cv], f)

In [29]:
print("Topics found via LDA:")
lda_topics(lda, cv, n_words=N_WORDS)

Topics found via LDA:


['chuj kurwa chuja jesteś chyba dupe wie masz tyle czas',
 'kurwa xd wiem gt chyba pis chce chuja proszę walić',
 'robić kurwa xd chyba chce wiem serio da raz czas',
 'xd chce mieć jebac chuj dupy wiem pis chyba ni',
 'zrobić kobiet pis robić ludzie polaków prostu czas mamy tyłek',
 'dupy ludzi bym jutro trzy zaraz chyba xd takich strasz',
 'ludzie dupie łeb pis chce wrzód chodzi zamiast ludzi takim',
 'kurwa dupy chuj chce ludzie wiem zaraz mac domu kurwie',
 'kurwy będziecie dupę masz kurwa dupe powoli ludzi chyba potem',
 'kurwa jesteś tez tyle 13 chyba piątek ludzi dalej wiem']